# Written by Max Paul
# April 18, 2021
# Cleaning data to get only the top shot username from the tweets
# Then pushing the final table back to the SQL data base
# experimenting in automating this entire process

### Connecting to my data base

In [278]:
import psycopg2
conn = psycopg2.connect(host="localhost",
                        database="top-twit-mapping",
                        port=5432,
                        user='postgres',
                        password=3301)

### Querying the database to only select the tweets that contain 'TS"
#### Also transforming the data into a dataframe for operations and datacleaning

In [279]:
import pandas as pd

# creating an instance of an operation
cur = conn.cursor()

# our command we want to send to the database
command =   '''
            select user_id,tweet
            from twitter_tweet
            where to_tsvector('english',tweet) @@ plainto_tsquery('english','TS')
            '''

# executing the command
cur.execute(command)

# extracting all the data
tupples = cur.fetchall()
# closing this connection
cur.close()
# col names
column_names = ['user_id','tweet']
# to a dataframe
data = pd.DataFrame(tupples, columns=column_names)

len(data)

492

# Preprocessing the data for username extraction

In [280]:
import re
def preprocess(dataframe,tweet):

    # remove links
    dataframe[tweet] = dataframe[tweet].str.replace(r'http\S+', '')
    # remove mentions
    dataframe[tweet] = dataframe[tweet].str.replace(r'@\w+', '')
    # remove multiple spaces
    dataframe[tweet] = dataframe[tweet].str.replace(r'\s+', '')
    # remove retweets
    dataframe[tweet] = dataframe[tweet].str.replace(r'RT\S+', '')
    return dataframe[tweet]


data['tweet'] = preprocess(data,'tweet')

<ipython-input-280-8955b68ed43f>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[tweet] = dataframe[tweet].str.replace(r'http\S+', '')
<ipython-input-280-8955b68ed43f>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[tweet] = dataframe[tweet].str.replace(r'@\w+', '')
<ipython-input-280-8955b68ed43f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[tweet] = dataframe[tweet].str.replace(r'\s+', '')
<ipython-input-280-8955b68ed43f>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe[tweet] = dataframe[tweet].str.replace(r'RT\S+', '')


In [281]:
data = pd.DataFrame(data)

In [282]:
data = data[data['tweet'] != '']

#### Data before name extraction

In [283]:
data.dtypes

user_id     int64
tweet      object
dtype: object

#### TS NAME extraction

In [284]:
data['tweet'] = data['tweet'].str.split(':|-.').str[1]

# FINAL USER ID ALONG WITH THEIR TOPSHOT USER NAME FROM THE TWEET LETS GOOOOO

### PUSHING TO THE DATABASE

- notes, cant just call the whole column to insert you have to do it row by row

In [302]:
conn = psycopg2.connect(host="localhost",
                        database="top-twit-mapping",
                        port=5432,
                        user='postgres',
                        password=3301)

In [304]:
def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [307]:
data

,user_id,tweet
2,1534709353,sexbobombtastic
5,104336723,acNutZ
8,2786668408,bradurday
9,1372501704021835777,and1mixtapeofficial
10,1354023153538723840,okallday
...,...,...
483,1086146383705296896,AgeNNortoN
484,1146646709629440000,Katipo
485,1194687857253851136,NaN
486,596547734,KiuKa79


In [306]:
# checking that our push was successful!
# Inserting each row
for i in data.index:
    query = '''INSERT INTO topshot_username (user_id,ts_username) VALUES (%s,%s) ON CONFLICT
                (user_id) DO NOTHING;''' % (data['user_id'], data['tweet'])
    single_insert(conn, query)
# Close the connection
conn.close()


                 ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username) VALUES (2               1534709353
                                                             ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username) VALUES (2               1534709353
                                                             ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username) VALUES (2               1534709353
                                                             ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username) VALUES (2               1534709353
                                                             ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username) VALUES (2               1534709353
                                                             ^

Error: syntax error at or near "1534709353"
LINE 1: ...ame (user_id,ts_username)